In [2]:
from numba import cuda
import numpy as np 
import math



In [3]:
x_host = np.ones(shape=(1048575))

In [4]:
def host_inc_one(arr):
    for i  in range(arr.shape[0]):
        arr[i] += 1

In [ ]:
%%timeit
host_inc_one(x_host)

In [ ]:
@cuda.jit()
def device_inc_one(arr):
    pos = cuda.grid(1)
    if pos < arr.size:
        arr[pos] += 1


x_device = cuda.to_device(x_host)
print(x_device.size)
threadsperblock = 256
blockspergrid = (x_device.size + (threadsperblock - 1)) // threadsperblock

In [ ]:
%%timeit
device_inc_one[blockspergrid,threadsperblock](x_device)

In [ ]:
x_device.copy_to_host()

In [9]:
x_device = cuda.to_device(np.ones(shape=(256,256)))

In [10]:
@cuda.jit
def device_inc_one_2d(arr):
    x,y = cuda.grid(2)
    if(x<arr.shape[0] and y<arr.shape[1]):
        arr[x,y] += 1

In [11]:
threadsperblock = (8,8)
blockspergrid_x = math.ceil(x_device.shape[0]/threadsperblock[0])
blockspergrid_y = math.ceil(x_device.shape[1]/threadsperblock[1])
blockspergrid = (blockspergrid_x,blockspergrid_y)



In [ ]:
%%timeit
device_inc_one_2d[blockspergrid,threadsperblock](x_device)

In [ ]:
b = [[1,2,3],[4,5,6],[7,8,9]]

for obj in b[0]:
    obj+=1
    print(obj)

print(b)

In [ ]:
from numba import cuda
import os
# 設置 CUDA 可見設備
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 列出所有可用的 GPU
print("Available GPUs:", cuda.gpus)

# 嘗試選擇 GPU 1
try:
    cuda.select_device(1)
    print("Selected GPU 1")
except cuda.cudadrv.error.CudaSupportError as e:
    print("Failed to select GPU 1:", e)

# 確認當前選擇的 GPU
print(cuda.get_current_device())

In [ ]:
config_arr = [30, 0.3, 0.1, 1]
x = 1
r = 5
(-x/(r**2)) * config_arr[0]*config_arr[3]

In [1]:
import numpy as np
from numba import cuda
import math
import concurrent.futures

@cuda.jit
def update_cuda(board):
    x,y = cuda.grid(2)
    r = ((0-x)**2 + (0-y)**2)
    if(r < 25):
        board[x][y][0] += 1
        board[x][y][1] += 1


In [15]:

mother_board = np.zeros(shape=(800,600,7),dtype='float32') # 800x600 x 每單位資料(force x, force y, velocity x, velocity y)
device_board = cuda.to_device(mother_board)
threadsperblock = (32,32)
blockspergrid_x = math.ceil(device_board.shape[0]/threadsperblock[0])
blockspergrid_y = math.ceil(device_board.shape[1]/threadsperblock[1])
blockspergrid = (blockspergrid_x,blockspergrid_y)

x =  [i for i in range(500)]
for _ in range(500):
    update_cuda[blockspergrid,threadsperblock](device_board)
mother_board = device_board.copy_to_host()
print(mother_board)

[[[     0.       0.       0.  ...      0.       0.       0. ]
  [     0.       0.     500.  ...      0.  -50000.     500. ]
  [     0.       0.     500.  ...      0.   -6250.     500. ]
  ...
  [     0.       0.       0.  ...      0.       0.       0. ]
  [     0.       0.       0.  ...      0.       0.       0. ]
  [     0.       0.       0.  ...      0.       0.       0. ]]

 [[     0.       0.     500.  ... -50000.       0.     500. ]
  [     0.       0.     500.  ... -12500.  -12500.     500. ]
  [     0.       0.     500.  ...  -2000.   -4000.     500. ]
  ...
  [     0.       0.       0.  ...      0.       0.       0. ]
  [     0.       0.       0.  ...      0.       0.       0. ]
  [     0.       0.       0.  ...      0.       0.       0. ]]

 [[     0.       0.     500.  ...  -6250.       0.     500. ]
  [     0.       0.     500.  ...  -4000.   -2000.     500. ]
  [     0.       0.     500.  ...  -1562.5  -1562.5    500. ]
  ...
  [     0.       0.       0.  ...      0.       

In [22]:

mother_board = np.zeros(shape=(800,600,7),dtype='float32') # 800x600 x 每單位資料(force x, force y, velocity x, velocity y)
device_board = cuda.to_device(mother_board)
threadsperblock = (32,32)
blockspergrid_x = math.ceil(device_board.shape[0]/threadsperblock[0])
blockspergrid_y = math.ceil(device_board.shape[1]/threadsperblock[1])
blockspergrid = (blockspergrid_x,blockspergrid_y)
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(update_cuda[blockspergrid,threadsperblock](device_board), [i for i in range(500)]) 
for _ in range(500):
    update_cuda[blockspergrid,threadsperblock](device_board)
mother_board = device_board.copy_to_host()


26.6 ms ± 895 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


True
